<a href="https://colab.research.google.com/github/placeasmedia/text_classification/blob/master/text_as_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import dependencies


In [4]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *

# Load data

In [5]:
def load_data(filename):
    df = pd.read_csv(filename)
    selected = ['text']
    non_selected = list(set(df.columns) - set(selected))

    df = df.drop(non_selected, axis=1)
    df = df.dropna(axis=0, how='any', subset=selected)
    df = df.reindex(np.random.permutation(df.index))

#     labels = df[selected[0]].tolist()

#     x_raw= df[selected[0]].apply(lambda x: seg_char(x).split(' ')).tolist() 
    x_raw= df[selected[0]]

    return x_raw

# local data directory

In [9]:
!ls
# X1 = load_data("./censored_weiboscope.csv")
# X2 = load_data("./uncensored_weiboscope.csv")
# Y1 = np.ones(len(X1), dtype=int)
# Y2 = np.zeros(len(X2), dtype=int)
# df1 = pd.DataFrame(X1)
# df2 = pd.DataFrame(X2)
# df = pd.concat([df1, df2])
# X = np.array(df)
# y = np.concatenate((Y1, Y2))
# df['label'] = y


sample_data
